In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [ ]:
PATHS = [
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
    '../input/cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'
    
]
df = pd.read_csv(PATHS[len(PATHS)-1])


In [ ]:
df[' Label'].value_counts()

In [ ]:
# for i in range(0,len(PATHS)):
#     temp = pd.read_csv(PATHS[i])
#     print(PATHS[i])
#     print(temp[' Label'].value_counts())
#     print('\n')
    

In [ ]:
m = df.loc[df[' Flow Packets/s'] != np.inf,' Flow Packets/s'].max()
df[' Flow Packets/s'].replace(np.inf,m,inplace=True)
m = df.loc[df['Flow Bytes/s'] != np.inf,'Flow Bytes/s'].max()
df['Flow Bytes/s'].replace(np.inf,m,inplace=True)

In [ ]:
dtypes = df.dtypes
print(f"Number of columns with Int {len(dtypes[dtypes == int])}")
print(f"Number of columns with float {len(dtypes[dtypes == float])}")
print(f"Number of columns with object {len(dtypes[dtypes == object])}")

In [ ]:
null_index = df[df['Flow Bytes/s'].isnull()].index.tolist()
targets = df.loc[null_index,' Label']

In [ ]:
null_values = df.isna().sum()
null_values[null_values >0]

In [ ]:
null_index = np.where(df['Flow Bytes/s'].isnull())[0]
df.dropna(inplace = True)

In [ ]:
# labels = df[' Label'].unique()
# label_enc  = dict()
# for i in range(len(labels)):
#     label_enc[labels[i]] = i
# df[' Label'] = df[' Label'].map(label_enc)
# import plotly.express as px
# fig = px.imshow(df.corr())
# fig.show()


In [ ]:
# import plotly.express as px
# fig = px.imshow(df.corr())
# fig.show()

In [ ]:
# temp = df[df[' Label'] == 'BENIGN']
# temp[' Destination Port'].describe()
# temp = temp.sample(frac = 0.1)

In [ ]:
# df = df[df[' Label'] != 'BENIGN']
# df = pd.concat([df,temp])

In [ ]:
from sklearn.model_selection import StratifiedKFold
df['folds'] = 0
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
for i, (_, test_index) in enumerate(skf.split(df[[' Destination Port']], df[' Label'])):
    df.iloc[test_index, -1] = i

In [ ]:
df = pd.get_dummies(df)

In [ ]:
col = [' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count',
       ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count',
       ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count',
       ' ECE Flag Count', ' Down/Up Ratio', ' Average Packet Size',
       ' Avg Fwd Segment Size', ' Avg Bwd Segment Size',
       ' Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk',
       ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk',
       'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', ' Subflow Fwd Bytes',
       ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward',
       ' Init_Win_bytes_backward', ' act_data_pkt_fwd',
       ' min_seg_size_forward', 'Active Mean', ' Active Std', ' Active Max',
       ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min']

In [ ]:
train_df= df[df['folds'] != 5]
valid_df = df[df['folds'] == 5]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_df[col] = scaler.fit_transform(train_df[col])
valid_df[col] = scaler.transform(valid_df[col])

In [ ]:
# train_df[' Label'].value_counts()

In [ ]:
# train_df[' Label'] = train_df[' Label'].map({'DDoS':0, 'BENIGN':1})
# valid_df[' Label'] = valid_df[' Label'].map({'DDoS':0, 'BENIGN':1})

# Model 

In [ ]:
def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, ls = 1e-2, lr = 1e-3):
    inp = tf.keras.layers.Input(shape = (num_columns, ))
    x = tf.keras.layers.BatchNormalization()(inp)
    #Multi Layer perceptron
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rates[3])(x)
    
    for i in range(2, len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('swish')(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 2])(x)
    
    out = tf.keras.layers.Dense(num_labels, activation = 'sigmoid', name = 'action')(x)
#     else:    
#         out = tf.keras.layers.Dense(num_labels, activation = 'sigmoid', name = 'action')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs = [out])
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
                  loss = {
                          'action': tf.keras.losses.BinaryCrossentropy(label_smoothing = ls), 
                         },
                  metrics = { 
                             'action': tf.keras.metrics.Accuracy(name = 'accuracy'), 
                            }, 
                 )
    
    return model

In [ ]:
params = {'num_columns': len(col), 
          'num_labels': 2, 
          'hidden_units': [96, 96, 896, 448, 448, 256], 
          'dropout_rates': [0.03527936123679956, 0.038424974585075086, 0.42409238408801436, 0.10431484318345882, 0.49230389137187497, 0.32024444956111164, 0.2716856145683449, 0.4379233941604448], 
          'ls': 0, 
          'lr':1e-3, 
         }

In [ ]:
batch_size = 64

In [ ]:
train_df.iloc[:,-2:]

In [ ]:
fold = 5
ckp_path = f'JSModel_{fold}.hdf5'
model = create_mlp(**params)
ckp = ModelCheckpoint(ckp_path, monitor = 'val_accuracy', verbose = 0, 
                      save_best_only = True, save_weights_only = True, mode = 'max')
es = EarlyStopping(monitor = 'val_accuracy', min_delta = 1e-4, patience = 20, mode = 'max', 
                   baseline = None, restore_best_weights = True, verbose = 0)
history = model.fit(train_df[col], [train_df.iloc[:,-2:],train_df.iloc[:,-2:]], 
                    validation_data = (valid_df[col], [ valid_df.iloc[:,-2:],valid_df.iloc[:,-2:]]), 
                    epochs = 10, batch_size = batch_size, callbacks = [ckp, es], verbose = True)
hist = pd.DataFrame(history.history)

In [ ]:
model.load_weights('../input/ann-cicids/JSModel_5.hdf5')

In [ ]:
res = model.predict(valid_df[col])
res = np.argmax(res,axis =1)

truth = np.argmax(valid_df.iloc[:,-2:].values,axis = 1)

truth = np.argmax(valid_df.iloc[:,-2:].values,axis = 1)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(truth, res)
precision,recall,_,_ = precision_recall_fscore_support(truth,res)


In [ ]:
epochs = [ i for i in range(1,11)]
hist['epochs'] = epochs

In [ ]:
import plotly.graph_objects as go
import json
!pip install -U kaleido
import kaleido

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                            x=hist['epochs'], y=hist['val_loss'],
                            mode='lines+markers',
                            name = "Validation Loss"
                        )
            )

fig.add_trace(go.Scatter(
                            x=hist['epochs'], y=hist['loss'],
                            mode='lines+markers',
                            name = "Train Loss"
                        )
            )

fig.update_layout(title="Loss Versus Epochs for MLP",
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.write_image("mlp_loss.jpeg")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                            x=hist['epochs'], y=hist['val_AUC'],
                            mode='lines+markers',
                            name = "Validation AUC"
                        )
            )

fig.add_trace(go.Scatter(
                            x=hist['epochs'], y=hist['AUC'],
                            mode='lines+markers',
                            name = "Train AUC"
                        )
            )
fig.update_layout(title="AUC Versus Epochs for MLP",
                   xaxis_title='Epoch',
                   yaxis_title='AUC')
fig.write_image("mlp_auc.jpeg")
fig.show()